# Estimator

The Estimator primitive service on Qiskit Runtime lets you efficiently calculate and interpret expectation values of quantum operators required for many algorithms. You can specify a list of circuits and observables, then evaluate expectation values and variances for a given parameter input.

`qiskit-ibm-runtime` now provides a simple `IBMEstimator` class which allows access to the Estimator primitive service.

Instantiate the `IBMRuntimeService` from one of the saved accounts like below.

In [1]:
from qiskit_ibm_runtime import IBMRuntimeService

# Save cloud account on disk.
# IBMRuntimeService.save_account(auth="cloud", token=<IBM Cloud API key>, instance=<IBM Cloud CRN> or <IBM Cloud service name>)

# Or save legacy account on disk.
# IBMRuntimeService.save_account(auth="legacy", token=<IBM Quantum API key>)

service = IBMRuntimeService()

Instantiate the `IBMEstimator` class by (optionally) passing the `service` instance instantiated above and also an optional backend name or `IBMBackend` object. 

Note:
- If `service` parameter is not set, it defaults to `IBMRuntimeService()` which means it will use your saved `cloud` or `legacy` accounts (in that order).
- If `backend` parameter is not set, then a least busy backend is automatically selected (IBM Cloud only).

In [2]:
from qiskit_ibm_runtime import IBMEstimator

estimator_factory = IBMEstimator(service=service, backend="ibmq_qasm_simulator")

Instantiating `IBMEstimator` returns a factory which can be used to open a session with the Estimator primitive service and pass an initial list of `circuits`, `observables` and `parameters` (optional). Let's prepare the quantum circuits and observables as shown below.

In [3]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

print(psi1.draw())
print(psi2.draw())

     ┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) │
q_1: ┤1                                               ├
     └────────────────────────────────────────────────┘
     ┌──────────────────────────────────────────────────────────┐
q_0: ┤0                                                         ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
q_1: ┤1                                                         ├
     └──────────────────────────────────────────────────────────┘


The quantum circuits and observables can then be passed as lists to the `estimator_factory`, which instantiates and opens a session with the Estimator primitive service and returns a handle to the session, `estimator`. `estimator` is an instance of `Estimator` class in `qiskit-ibm-runtime` which implements the `BaseEstimator` interface in `qiskit-terra`.

`estimator` can then be called multiple times to write different `circuit_indices` (a list of circuit indices), `observable_indices` (a list of observable indices) and `parameter_values` (an optional list of concrete parameters to be bound) to the session and read the calculated expectation values. The result is an instance of `EstimatorResult` class in `qiskit-terra` and it contains expectation values and metadata.

In [4]:
with estimator_factory(circuits=[psi1, psi2], observables=[H1, H2, H3]) as estimator:
    # parameter values
    theta1 = [0, 1, 1, 2, 3, 5]
    theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
    theta3 = [1, 2, 3, 4, 5, 6]

    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    result1 = estimator(circuit_indices=[0], observable_indices=[0], parameter_values=[theta1])
    print(result1)

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    result2 = estimator(circuit_indices=[0, 0], observable_indices=[1, 2], parameter_values=[theta1]*2)
    print(result2)

    # calculate [ <psi2(theta2)|H2|psi2(theta2)> ]
    result3 = estimator(circuit_indices=[1], observable_indices=[1], parameter_values=[theta2])
    print(result3)

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>, <psi1(theta3)|H1|psi1(theta3)> ]
    result4 = estimator(circuit_indices=[0, 0], observable_indices=[0, 0], parameter_values=[theta1, theta3])
    print(result4)

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>,
    #             <psi2(theta2)|H2|psi2(theta2)>,
    #             <psi1(theta3)|H3|psi1(theta3)> ]
    result5 = estimator(circuit_indices=[0, 1, 0], observable_indices=[0, 1, 2], parameter_values=[theta1, theta2, theta3])
    print(result5)

EstimatorResult(values=array([1.55078125]), metadata=[{'variance': 9.184310913085938, 'shots': 1024}])
EstimatorResult(values=array([-0.57226562,  0.09960938]), metadata=[{'variance': 0.6725120544433594, 'shots': 1024}, {'variance': 1.9837303161621094, 'shots': 1024}])
EstimatorResult(values=array([0.18554688]), metadata=[{'variance': 0.9655723571777344, 'shots': 1024}])
EstimatorResult(values=array([1.546875  , 1.06835938]), metadata=[{'variance': 9.175872802734375, 'shots': 1024}, {'variance': 12.271556854248047, 'shots': 1024}])
EstimatorResult(values=array([ 1.55078125,  0.19921875, -1.13085938]), metadata=[{'variance': 9.120590209960938, 'shots': 1024}, {'variance': 0.9603118896484375, 'shots': 1024}, {'variance': 1.2225608825683594, 'shots': 1024}])


In [5]:
from qiskit.tools.jupyter import *

%qiskit_copyright